In [1]:
import os

import deepl
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
import tiktoken
from openai import OpenAI
import faiss
import numpy as np


# Enable tqdm for pandas
tqdm.pandas()

In [3]:
load_dotenv("./../.env")

True

In [7]:
index = faiss.read_index('../datasets/math-500-uk.index')
inference_df = pd.read_csv('../datasets/math-500-uk-inference.csv')

In [4]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
client = OpenAI(api_key=openai_api_key)

def get_embedding(text, model="text-embedding-ada-002"):
    response = client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

model = "text-embedding-ada-002"

In [11]:
def similarity_search(query: str, top_k: int, similarity_threshold: float = 0.7) -> pd.DataFrame:
    query_embedding = np.array(get_embedding(query, model=model)).astype('float32').reshape(1, -1)

    # Normalize the query embedding
    faiss.normalize_L2(query_embedding)

    # Perform the search to retrieve top k similar embeddings
    distances, indices = index.search(query_embedding, top_k)

    # Convert distances to cosine similarities
    # Since embeddings are normalized, the inner product is equivalent to cosine similarity
    cosine_similarities = distances[0]


    # Filter results based on the similarity threshold
    filtered_indices = indices[0][cosine_similarities >= similarity_threshold]

    # Retrieve corresponding texts
    similar_texts = inference_df.iloc[filtered_indices]

    return similar_texts

similar_examples_df = similarity_search(
    query="Задача по геометрії",
    top_k=25,
    similarity_threshold=0.75
)
similar_examples_df

,problem,solution,answer,subject,level,unique_id,original_level,problem_char_count,solution_char_count,subject_char_count,...,problem_translated,solution_translated,subject_translated,level_translated,problem_translated_token_count,solution_translated_token_count,subject_translated_token_count,level_translated_token_count,embedding_text,embedding
222,The area of $\triangle ABC$ is 6 square centim...,"Since $AB \parallel DE,$ we know that $\angle ...",54,Geometry,High-Level Math Competition Problems; Regional...,test/geometry/547.json,4,342,424,8,...,Площа $\трикутника ABC$ дорівнює 6 квадратних ...,"Оскільки $AB \паралельний DE,$ ми знаємо, що $...",Геометрія,Математичні олімпіадні задачі високого рівня; ...,176,272,7,50,Математичні олімпіадні задачі високого рівня; ...,"[0.010712559334933758, -0.007907348684966564, ..."
308,Let $ABCD$ be a regular tetrahedron with side ...,The plane intersects each face of the tetrahed...,1+2\sqrt{3},Geometry,Advanced Math Competitions; International Olym...,test/geometry/711.json,5,203,375,8,...,Нехай $ABCD$ - правильний тетраедр зі стороною...,Площина перетинає кожну грань тетраедра по сер...,Геометрія,Поглиблені математичні змагання; міжнародна ол...,117,191,7,56,Поглиблені математичні змагання; міжнародна ол...,"[0.008417361415922642, 0.000720419455319643, 5..."
299,"Consider the rectangle with vertices at $(5,4)...",Points with integer coordinates are called lat...,63,Geometry,High-Level Math Competition Problems; Regional...,test/geometry/1097.json,4,156,387,8,...,"Розглянемо прямокутник з вершинами $(5,4),$ $(...",Точки з цілочисельними координатами називаютьс...,Геометрія,Математичні олімпіадні задачі високого рівня; ...,85,209,7,50,Математичні олімпіадні задачі високого рівня; ...,"[-0.0031878254376351833, -0.003637160174548626..."
169,What is the number of square units in the area...,"This triangle is isosceles, and so the altitud...",12,Geometry,Foundational High-School Math,test/geometry/802.json,2,98,337,8,...,Скільки квадратних одиниць міститься у площі т...,"Цей трикутник рівнобедрений, тому висота, пров...",Геометрія,Фундаментальна математика для середньої школи,62,225,7,23,Фундаментальна математика для середньої школи\...,"[0.021239973604679108, 0.022146791219711304, 0..."
123,"In the diagram, $D$ and $E$ are the midpoints ...",$\triangle DBC$ has base $\overline{BC}$ of le...,8,Geometry,High-Level Math Competition Problems; Regional...,test/geometry/283.json,4,568,587,8,...,На рисунку $D$ і $E$ - середини відрізків $\ov...,Трикутник $\triangle DBC$ має основу $\overlin...,Геометрія,Математичні олімпіадні задачі високого рівня; ...,271,318,7,50,Математичні олімпіадні задачі високого рівня; ...,"[0.008777080103754997, -0.00464252894744277, -..."
152,"The medians $AD$, $BE$, and $CF$ of triangle $...","Since $E$ is the midpoint of $AC$, the area of...",8,Geometry,Advanced Math Competitions; International Olym...,test/geometry/172.json,5,260,873,8,...,"Медіани $AD$, $BE$ і $CF$ трикутника $ABC$ пер...","Оскільки $E$ є серединою $AC$, то площа трикут...",Геометрія,Поглиблені математичні змагання; міжнародна ол...,145,453,7,56,Поглиблені математичні змагання; міжнародна ол...,"[0.010307596065104008, -0.0026040917728096247,..."
154,"A gecko is in a room that is 12 feet long, 10 ...",[asy]\nimport three;\ncurrentprojection=orthog...,2\sqrt{113},Geometry,Advanced Math Competitions; International Olym...,test/geometry/880.json,5,546,3356,8,...,"Гекон знаходиться у кімнаті довжиною 12 футів,...",[asy]\nімпортувати три;\ncurrentprojection=ort...,Геометрія,Поглиблені математичні змагання; міжнародна ол...,329,1887,7,56,Поглиблені математичні змагання; міжнародна ол...,"[-0.005317579489201307, 0.01098331343382597, -..."
100,A hexagon is inscribed in a circle: [asy]\npai...,"Labeling our vertices will help a great deal, ...",145^\circ,Geometry,Advanced Math Competitions; International Olym...,test/geometry/183.json,5,409,1259,8,...,"У коло вписано шестикутник: [asy].\nпара pA, p...","Позна

In [24]:
from ipywidgets import Image
import re

example_formatting = """
Тематика:
{subject}

Проблема:
{problem}

Вирішення:
{solution}

Відповідь:
{answer}

"""
from IPython.display import display, Markdown
similar_examples_df.loc[:, "example_text"] = similar_examples_df.apply(lambda row: example_formatting.format(
    subject=row["subject_translated"],
    problem=row["problem_translated"],
    solution=row["solution_translated"],
    answer=row["answer"]
), axis=1)

for example in similar_examples_df.loc[:, "example_text"]:

    example_text = re.sub(r'\[asy\].*?\[/asy\]', '', example, flags=re.DOTALL)

    display(Markdown(example_text))


Тематика:
Геометрія

Проблема:
Площа $\трикутника ABC$ дорівнює 6 квадратних сантиметрів. $\overline{AB}\|\overline{DE}$. $BD=4BC$. Скільки квадратних сантиметрів у площі $\трикутника CDE$? 

Вирішення:
Оскільки $AB \паралельний DE,$ ми знаємо, що $\кут A = \кут E$ і $\кут B = \кут D.$ Це чудово працює, оскільки це означає, що $\трикутник ABC \sim EDC.$ Якщо $BD = 4BC,$ це означає, що $CD = BD - BC = 3BC.$ Отже, відношення сторін у $ABC$ до $EDC$ дорівнює $1:3,$ а це означає, що відношення їх площ дорівнює $1:9.

Оскільки площа $\трикутника ABC$ дорівнює $6\text{cm}^2,$ то площа $\трикутника CDE$ дорівнює $\boxed{54}\text{cm}^2.$

Відповідь:
54




Тематика:
Геометрія

Проблема:
Нехай $ABCD$ - правильний тетраедр зі стороною 2. Площина, паралельна ребрам $AB$ і $CD$ і розташована на півдорозі між ними, розрізає $ABCD$ на дві частини. Знайдіть площу поверхні однієї з цих частин.

Вирішення:
Площина перетинає кожну грань тетраедра по середній лінії грані; з симетрії випливає, що у перетині площини з тетраедром утворюється квадрат зі стороною 1. Площа кожної фігури дорівнює половині загальної площі тетраедра плюс площа квадрата, тобто $\frac{1}{2}\cdot 4\cdot \frac{2^2 \sqrt{3}}{4}+1=\boxed{1+2\sqrt{3}}$.

Відповідь:
1+2\sqrt{3}




Тематика:
Геометрія

Проблема:
Розглянемо прямокутник з вершинами $(5,4),$ $(5,-4),$ $(-5,4),$ $(-5,-4)$. Скільки цілочисельних координат буде знаходитись строго всередині прямокутної області?

Вирішення:
Точки з цілочисельними координатами називаються точками решітки. Довжина прямокутника дорівнює $5 - (-5) = 10$ одиниць. Між двома вертикальними сторонами прямокутника буде 9 позицій решітки. Висота прямокутника дорівнює $4 - (-4) = 8$ одиниць. Між верхньою і нижньою сторонами прямокутника буде 7 позицій решітки. Всього буде $9 \times 7 = \boxed{63}$ точок решітки.

Відповідь:
63




Тематика:
Геометрія

Проблема:
Скільки квадратних одиниць міститься у площі трикутника, сторони якого дорівнюють 5, 5 і 6 одиницям?

Вирішення:
Цей трикутник рівнобедрений, тому висота, проведена до сторони з довжиною 6, повинна потрапити на цю сторону в її середині. Таким чином, наш трикутник розбивається на два прямокутних трикутника з гіпотенузою $5$ і однією стороною довжиною $3$. Таким чином, кожен з цих трикутників є трикутником $3-4-5$, і кожен з них має площу $\frac{3 \times 4}{2} = 6$, а загальна площа дорівнює $\boxed{12}$.

Відповідь:
12




Тематика:
Геометрія

Проблема:
На рисунку $D$ і $E$ - середини відрізків $\overline{AB}$ і $\overline{BC}$ відповідно.  Знайдіть площу чотирикутника $DBEF$. 

Вирішення:
Трикутник $\triangle DBC$ має основу $\overline{BC}$ довжини 8 і висоту $\overline{BD}$ довжини 3, тому його площа дорівнює $\frac{1}{2}\times8\times 3=12$.

Площа чотирикутника $DBEF$ дорівнює площі $\трикутника DBC$ мінус площа $\трикутника FEC$.

Трикутник $\triangle FEC$ має основу $EC=BC-BE=8-4=4$. Висота $\трикутника FEC$ дорівнює відстані по вертикалі від точки $F$ до осі $x$, яка дорівнює координаті $y$ точки $F$, або 2. Отже, площа $\трикутника FEC$ дорівнює $\frac{1}{2}\times4\times 2=4$.

Нарешті, площа чотирикутника $DBEF$ дорівнює $12-4=\boxed{8}$.

Відповідь:
8



...


Тематика:
Геометрія

Проблема:
Трикутник $\triangle ABC$ з $\angle A = 14^\circ$ вписано в коло, де $AB$ - діаметр. Скільки градусів у $\куті B$?

Вирішення:
Якщо $AB$ є діаметром, то це означає, що трикутник повинен мати прямий кут при $C.$ Отже, маємо, що \begin{align*}
\кут B &= 180^\circ - (\кут A + \кут C) \\
&= 180^\circ - (14^\circ + 90^\circ) = \boxed{76^\circ}.
\end{align*}

Відповідь:
76^\circ




Тематика:
Преалгебра

Проблема:
Довжини двох протилежних сторін квадрата зменшили на $40\%$, а довжини двох інших сторін збільшили на $50\%$, утворивши прямокутник. На скільки відсотків зменшиться площа квадрата?

Вирішення:
Нехай $A$ - площа квадрата. Довжини однієї пари протилежних сторін зменшили на $40\%$, так що площа стала $.6A$. Другу пару сторін збільшили на $50\%$, так що площа стала $1.5\cdot .6 A = .9A$. Таким чином, площа зменшилась на $\boxed{10}$ відсотків.

Відповідь:
10




Тематика:
Геометрія

Проблема:
Паралелограм, обмежений прямими $y=ax+c$, $y=ax+d$, $y=bx+c$ і $y=bx+d$, має площу 18. Паралелограм, обмежений прямими $y=ax+c$, $y=ax-d$, $y=bx+c$ і $y=bx-d$, має площу 72. Знаючи, що $a$, $b$, $c$ і $d$ - натуральні числа, знайдіть найменше можливе значення суми $a+b+c+d$.

Вирішення:
Дві вершини першого паралелограма знаходяться у точках $(0,c)$ та $(0,d)$.



Координати $x$ двох інших вершин паралелограма задовольняють рівняння $ax+c=bx+d$ і $ax+d=bx+c$, тому координати $x$ дорівнюють $\pm(c-d)/(b-a)$. Таким чином, паралелограм складається з двох трикутників, кожен з яких має площу \[
9=\frac{1}{2} \cdot |c-d| \cdot \left|\frac{c-d}{b-a}\right|.
\Звідси випливає, що $(c-d)^2=18|b-a|$.

За аналогічним міркуванням з використанням другого паралелограма, $(c+d)^2=72|b-a|$. Віднімання першого рівняння від другого дає $4cd=54|b-a|$, отже $2cd=27|b-a|$. Таким чином, $|b-a|$ парне, а $a+b$ мінімізується при $\{a,b\}=\{1,3\}$. Також $cd$ кратне 27, і $c+d$ мінімізується при $\{c,d\}=\{3,9\}$. Отже, найменше можливе значення $a+b+c+d$ дорівнює $1+3+3+9=\boxed{16}$. Зауважимо, що необхідні умови виконуються, коли $(a,b,c,d)=(1,3,3,9)$.

Відповідь:
16




Тематика:
Геометрія

Проблема:
Дві сторони трикутника мають довжину по $8$ одиниць. Якщо третя сторона має довжину цілого числа, який найбільший можливий периметр трикутника в одиницях?

Вирішення:
Нерівність трикутника говорить, що сума довжин будь-яких двох сторін повинна бути більшою за довжину третьої сторони. Це означає, що $8+8=16$ має бути більшою за довжину третьої сторони. Третя сторона має довжину цілого числа, тому найбільша можлива довжина дорівнює 15 одиницям. Отже, периметр $8+8+15=\boxed{31}$ одиниць.

Відповідь:
31




Тематика:
Геометрія

Проблема:
Яка довжина в одиницях радіуса сфери, об'єм і площа поверхні якої в кубічних і квадратних одиницях, відповідно, чисельно рівні?

Вирішення:
Об'єм кулі дорівнює $\frac{4}{3}\pi r^3$, а площа поверхні - $4\pi r^2$, тому
\[\frac{4}{3} \pi r^3 = 4 \pi r^2.\]Ми можемо поділити обидві сторони на $4 \pi r^2$, щоб отримати
\[\frac{1}{3} r = 1.\]Отже, $r = \boxed{3}.$

Відповідь:
3



We can see that DeepL over-translated some texts to uninterpretable LaTeX, so, we would prompt GPT to tackle this issue, and provide in its answer meaningful output.\
Moreover, we can do second-model verification approach, so the GPT would fix the examples translated with DeepL given translated and original text to preserve original context, I guess that would improve the dataset quality.
However, as we would see in the results, if we would provide nice and complete instructions on how to handle "unpurities" of the similarity context, the answers the GPT-4o-mini provides are quite impressive, so, it's a point to check whether the second-model verification would help us. I guess it'd allow us to simplify the system prompt and result in similar results. So it might be just unnecessary with this model. Nonetheless, it might be useful for smaller OpenSource LLMs, as they are high-dependent and very sensitive to input data.

In [33]:
system_prompt = """
Ти корисний помічник для генерації задач з математики українською мовою.
Маючи вхідний запит користувача ("USER_INPUT"), який включає тему, чи ключові слова, та знайдені схожі приклади задач ("SIMILAR_EXAMPLES") згенеруй 5 задач на тему, якщо користувач не попросить інакше.
Візьми до уваги схожі теми задач, та тему з питання якщо така задана.
Візьми до уваги потрібну складність задачі, якщо така задана. Якщо ж ні, спробуй згенерувати простіші завдання, вибираючи простіші приклади.
Текст у прикладах ("SIMILAR_EXAMPLES") може мати помилки у LaTeX синтаксисі після перекладу, спробуй їх виправити у відповіді, щоб згенерувати валідну відповідь.
Згенеруй 5 прикладів задач з покроковими розв’язками, уникаючи помилок.
Використовуй увесь доступний контекст для більшої точності та різноманітності.
Спробуй генерувати різноманітні задачі.
Якщо ви генеруєте код LaTeX або рівняння, використовуйте формат $ замість W( і $$ замість \\I; замість багаторядкового LaTeX у форматі
[
довільне рівняння
]
генеруй $$
довільне LaTeX рівняння для правильного трактування у Markdown форматі
$$
Також візьми до уваги, що при генерації коду чи рівняння LaTeX у однорядковому форматі, замість ( приклад LaTeX коду ) використовуй одинарний знак $ приклад LaTeX коду $

SIMILAR_EXAMPLES:
{examples}
"""

def generate_example(user_input: str):
    model = "gpt-4o-mini"

    similar_examples_df = similarity_search(
        query=user_input,
        top_k=15,
        similarity_threshold=0.75
    )

    similar_examples_df.loc[:, "example_text"] = similar_examples_df.apply(lambda row: example_formatting.format(
        subject=row["subject_translated"],
        problem=row["problem_translated"],
        solution=row["solution_translated"],
        answer=row["answer"]
    ), axis=1)
    examples_text = ""

    for example in similar_examples_df.loc[:, "example_text"]:
        examples_text += re.sub(r'\[asy\].*?\[/asy\]', '', example, flags=re.DOTALL)

    system_prompt_formatted = system_prompt.format(
        examples=examples_text
    )

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "developer", "content": system_prompt_formatted},
            {
                "role": "user",
                "content": user_input
            }
        ]
    )
    response = completion.choices[0].message.content

    display(Markdown(response))

generate_example("Привіт, згенеруй декілька простих задач по шкільній геометрії. Мені важко даються рівнобедрені трикутники.")

print()

/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/ipykernel_93999/2800089990.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_examples_df.loc[:, "example_text"] = similar_examples_df.apply(lambda row: example_formatting.format(


Привіт! Ось 5 простих задач по шкільній геометрії, зокрема про рівнобедрені трикутники, з покроковими розв'язками.

### Задача 1
**Проблема:** У рівнобедреному трикутнику $ABC$ сторони $AB$ і $AC$ рівні, і їх довжина дорівнює 6 см. Довжина основи $BC$ дорівнює 8 см. Знайдіть площу трикутника $ABC$.

**Вирішення:**
1. Складемо половину основи: $BM = MC = \frac{BC}{2} = \frac{8}{2} = 4 \text{ см}$.
2. Знайдемо висоту $h$ трикутника, використовуючи теорему Піфагора: 
   $$ AM^2 + BM^2 = AB^2 $$
   $$ h^2 + 4^2 = 6^2 $$
   $$ h^2 + 16 = 36 $$
   $$ h^2 = 20 $$
   $$ h = \sqrt{20} = 2\sqrt{5} \text{ см} $$
3. Площа $S$ трикутника:
   $$ S = \frac{1}{2} \cdot BC \cdot h = \frac{1}{2} \cdot 8 \cdot 2\sqrt{5} = 8\sqrt{5} \text{ см}^2 $$

**Відповідь:** $8\sqrt{5} \text{ см}^2$

---

### Задача 2
**Проблема:** В рівнобедреному трикутнику $DEF$, де $DE = DF$, довжина сторони $DE = 10$ см, а основа $EF = 12$ см. Яка висота, проведена з точки $D$ до сторони $EF$?

**Вирішення:**
1. Знайдемо половину основи: $EG = GF = \frac{EF}{2} = 6 \text{ см}$.
2. Визначимо висоту $h$:
   $$ DG^2 + EG^2 = DE^2 $$
   $$ h^2 + 6^2 = 10^2 $$
   $$ h^2 + 36 = 100 $$
   $$ h^2 = 64 $$
   $$ h = 8 \text{ см} $$

**Відповідь:** $8 \text{ см}$

---

### Задача 3
**Проблема:** У рівнобедреному трикутнику $GHI$ сторони $GH = GI$. Довжина сторін дорівнює 7 см, а основа $HI$ дорівнює 6 см. Знайдіть площу трикутника $GHI$.

**Вирішення:**
1. Знайдемо половину основи: $HJ = JI = 3 \text{ см}$.
2. Застосуємо теорему Піфагора:
   $$ GJ^2 + HJ^2 = GH^2 $$
   $$ h^2 + 3^2 = 7^2 $$
   $$ h^2 + 9 = 49 $$
   $$ h^2 = 40 $$
   $$ h = \sqrt{40} = 2\sqrt{10} \text{ см} $$
3. Площа:
   $$ S = \frac{1}{2} \cdot HI \cdot h = \frac{1}{2} \cdot 6 \cdot 2\sqrt{10} = 6\sqrt{10} \text{ см}^2 $$

**Відповідь:** $6\sqrt{10} \text{ см}^2$

---

### Задача 4
**Проблема:** У рівнобедреному трикутнику $JKL$ сторони $JK$ і $JL$ рівні та дорівнюють 5 см. Довжина основи $KL = 8$ см. Яка площа трикутника?

**Вирішення:**
1. Половина основи: $KM = ML = 4 \text{ см}$.
2. Знайдемо висоту:
   $$ JM^2 + KM^2 = JK^2 $$
   $$ h^2 + 4^2 = 5^2 $$
   $$ h^2 + 16 = 25 $$
   $$ h^2 = 9 $$
   $$ h = 3 \text{ см} $$
3. Площа трикутника:
   $$ S = \frac{1}{2} \cdot KL \cdot h = \frac{1}{2} \cdot 8 \cdot 3 = 12 \text{ см}^2 $$

**Відповідь:** $12 \text{ см}^2$

---

### Задача 5
**Проблема:** У рівнобедреному трикутнику $MNO$ сторони $MN = MO = 9$ см, а основа $NO = 10$ см. Знайдіть довжину висоти, проведеної з вершини до основи.

**Вирішення:**
1. Половина основи: $NP = PO = 5 \text{ см}$.
2. Визначаємо висоту:
   $$ MP^2 + NP^2 = MN^2 $$
   $$ h^2 + 5^2 = 9^2 $$
   $$ h^2 + 25 = 81 $$
   $$ h^2 = 56 $$
   $$ h = \sqrt{56} = 2\sqrt{14} \text{ см} $$

**Відповідь:** $2\sqrt{14} \text{ см}$

Сподіваюся, ці задачі будуть корисними для вас! Якщо потрібно більше, звертайтеся!

In [34]:
generate_example("Привіт, згенеруй декілька задач по інтегралах та похідних. Не бійся згенерувати важчі задачі, я хочу підготуватись до олімпіади.")

print()

/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/ipykernel_93999/2800089990.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_examples_df.loc[:, "example_text"] = similar_examples_df.apply(lambda row: example_formatting.format(


Звичайно! Ось 5 задач по інтегралах та похідних різної складності з покроковими розв'язками:

### Задача 1: Знайти похідну
Знайдіть похідну функції $$f(x) = 3x^4 - 5x^3 + 2x - 7.$$

**Розв'язок:**
1. Знайдемо похідну кожного члена функції:
   - Похідна від $3x^4$ дорівнює $12x^3$.
   - Похідна від $-5x^3$ дорівнює $-15x^2$.
   - Похідна від $2x$ дорівнює $2$.
   - Похідна від константи $-7$ дорівнює $0$.
2. Об'єднуємо похідні:
   $$f'(x) = 12x^3 - 15x^2 + 2.$$

**Відповідь:**
$$f'(x) = 12x^3 - 15x^2 + 2.$$

---

### Задача 2: Знайти інтеграл
Обчисліть інтеграл $$I = \int (4x^3 - 2x^2 + x - 5) \, dx.$$

**Розв'язок:**
1. Інтегруємо кожен член:
   - $\int 4x^3 \, dx = x^4$.
   - $\int -2x^2 \, dx = -\frac{2}{3}x^3$.
   - $\int x \, dx = \frac{1}{2}x^2$.
   - $\int -5 \, dx = -5x$.
2. Об'єднуємо результати та додаємо константу інтегрування $C$:
   $$I = x^4 - \frac{2}{3}x^3 + \frac{1}{2}x^2 - 5x + C.$$

**Відповідь:**
$$I = x^4 - \frac{2}{3}x^3 + \frac{1}{2}x^2 - 5x + C.$$

---

### Задача 3: Похідна складеної функції
Знайдіть похідну функції $$g(x) = \sin(2x^2 + 3).$$

**Розв'язок:**
1. Використовуємо правило ланцюга:
   - Нехай $u = 2x^2 + 3$, тоді $g(x) = \sin(u)$.
2. Знайдемо похідні:
   - $g'(x) = \cos(u) \cdot u'$.
   - $u' = 4x$.
3. Підставимо:
   $$g'(x) = \cos(2x^2 + 3) \cdot 4x = 4x \cos(2x^2 + 3).$$

**Відповідь:**
$$g'(x) = 4x \cos(2x^2 + 3).$$

---

### Задача 4: Інтеграл з тригонометричними функціями
Обчисліть інтеграл $$J = \int \cos^2(x) \, dx.$$

**Розв'язок:**
1. Застосуємо формулу зменшення:
   $$\cos^2(x) = \frac{1 + \cos(2x)}{2}.$$
2. Інтегруємо:
   $$J = \int \frac{1 + \cos(2x)}{2} \, dx = \frac{1}{2} \int 1 \, dx + \frac{1}{2} \int \cos(2x) \, dx.$$
3. Обчислимо:
   $$J = \frac{x}{2} + \frac{1}{2} \cdot \frac{1}{2} \sin(2x) + C = \frac{x}{2} + \frac{1}{4} \sin(2x) + C.$$

**Відповідь:**
$$J = \frac{x}{2} + \frac{1}{4} \sin(2x) + C.$$

---

### Задача 5: Похідна параметричної функції
Нехай $$x(t) = t^2 + 1, \quad y(t) = 2t^3 - 3t.$$ Знайдіть $\frac{dy}{dx}$.

**Розв'язок:**
1. Знайдемо похідні $x'(t)$ і $y'(t)$:
   - $x'(t) = 2t$.
   - $y'(t) = 6t^2 - 3$.
2. Знайдемо $\frac{dy}{dx}$ за формулою:
   $$\frac{dy}{dx} = \frac{y'(t)}{x'(t)} = \frac{6t^2 - 3}{2t}.$$
3. Спрощуємо:
   $$\frac{dy}{dx} = \frac{3t^2 - \frac{3}{2}}{t} = 3t - \frac{3}{2t}.$$

**Відповідь:**
$$\frac{dy}{dx} = 3t - \frac{3}{2t}.$$

---

Сподіваюсь, ці задачі виявляться корисними для вашої підготовки до олімпіади! Якщо потрібні додаткові задачі чи пояснення, будь ласка, дайте знати.

As we see, the answers look good and factual, the model does not struggle with the intense number of unclean data we gave (a point to improve).
For instance, if you look more thoroughly in our dataset, you'd be able to see that the data includes [ase] blocks in order to render visualizations from the asymptote package: https://artofproblemsolving.com/wiki/index.php/Asymptote:_Drawing
However, it's rather difficult to use this project in our rag-agent, as it's a burden to install, therefore, we just delete those blocks present in the original dataset, in order to minimally clean-up the retrieved similarity data to our prompt.
Also the instructions we've added on how to do proper markdown gave a huge boost, as DeepL sometimes over-translates as mentioned, and changes some commands like: \triangle to \трикутник which is not recognized by the LaTeX compiler, making our results look uglier.
Nonetheless, gpt-4o-mini is quite a powerful model, both in terms of following a large set of instructions we gave, a wast amount of examples with a lot of "garbage" (we did a minimal cleanup, and we can definitely improve and look into that more) and most importantly UKRAINIAN, this model is quite fluent in it, which is a huge plus in my opinion, whilst being reasonably cheap.